In [ ]:
# only if using colab
from google.colab import drive
drive.mount('/content/drive')
zip_path = "/content/drive/My Drive/dev_phase.zip"
!unzip "$zip_path" -d /content/


Mounted at /content/drive
Archive:  /content/drive/My Drive/dev_phase.zip
   creating: /content/subtask1/
   creating: /content/subtask1/dev/
  inflating: /content/subtask1/dev/nep.csv  
  inflating: /content/subtask1/dev/ita.csv  
  inflating: /content/subtask1/dev/pol.csv  
  inflating: /content/subtask1/dev/rus.csv  
  inflating: /content/subtask1/dev/tel.csv  
  inflating: /content/subtask1/dev/hin.csv  
  inflating: /content/subtask1/dev/hau.csv  
  inflating: /content/subtask1/dev/pan.csv  
  inflating: /content/subtask1/dev/ori.csv  
  inflating: /content/subtask1/dev/spa.csv  
  inflating: /content/subtask1/dev/deu.csv  
  inflating: /content/subtask1/dev/fas.csv  
  inflating: /content/subtask1/dev/arb.csv  
  inflating: /content/subtask1/dev/ben.csv  
  inflating: /content/subtask1/dev/amh.csv  
  inflating: /content/subtask1/dev/khm.csv  
  inflating: /content/subtask1/dev/tur.csv  
  inflating: /content/subtask1/dev/zho.csv  
  inflating: /content/subtask1/dev/eng.csv  
  i

# Bert baseline for POLAR

## Introduction

In this part of the starter notebook, we will take you through the process of all three Subtasks.

## Subtask 1 - Polarization detection

This is a binary classification to determine whether a post contains polarized content (Polarized or Not Polarized).

In [ ]:
langs = ["amh", "eng", "swa"]
LANG = langs[1]

## Imports

In [ ]:
import pandas as pd

from sklearn.metrics import recall_score, precision_score, f1_score
import numpy as np

import torch

from sklearn.metrics import f1_score

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
import wandb

# Disable wandb logging for this script
wandb.init(mode="disabled")

## Data Import

The training data consists of a short text and binary labels

The data is structured as a CSV file with the following fields:
- id: a unique identifier for the sample
- text: a sentence or short text
- polarization:  1 text is polarized, 0 text is not polarized

The data is in all three subtask folders the same but only containing the labels for the specific task.

In [ ]:
# Load the training and validation data for subtask 1

df = pd.read_csv(f'../data/dev_phase/subtask1/train/{LANG}.csv')

train, val = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

train.head()

,id,text,polarization
3000,eng_427cea503f2e3a2b4608d26fa87a55f0,The IDF needs some B52s.,0
366,eng_98eb4278a5fb9f249149a899b8f2c4e7,Fascinating life journey leads me to ANRAurora...,0
1965,eng_0d9c500e39edc99af8c61c45db8825a2,Lazy woke excuse to justify sjw practices,1
29,eng_69ebe50510087fb8c06cd12283a88ef8,5 takeaways on Republicans first impeachment h...,0
2689,eng_787ec0b4ddda46bbb5a0bf507a31d484,And yet Dems are supposed to be the breeders o...,1


# Dataset
-  Create a pytorch class for handling data
-  Wrapping the raw texts and labels into a format that Huggingface’s Trainer can use for training and evaluation

In [ ]:
# Fix the dataset class by inheriting from torch.utils.data.Dataset
class PolarizationDataset(torch.utils.data.Dataset):
  def __init__(self,texts,labels,tokenizer,max_length =128):
    self.texts=texts
    self.labels=labels
    self.tokenizer= tokenizer
    self.max_length = max_length # Store max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self,idx):
    text=self.texts[idx]
    label=self.labels[idx]
    encoding=self.tokenizer(text,truncation=True,padding=False,max_length=self.max_length,return_tensors='pt')

    # Ensure consistent tensor conversion for all items
    item = {key: encoding[key].squeeze() for key in encoding.keys()}
    item['labels'] = torch.tensor(label, dtype=torch.long)
    return item

Now, we'll tokenize the text data and create the datasets using `bert-base-uncased` as the tokenizer.

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Create datasets
train_dataset = PolarizationDataset(train['text'].tolist(), train['polarization'].tolist(), tokenizer)
val_dataset = PolarizationDataset(val['text'].tolist(), val['polarization'].tolist(), tokenizer)

Next, we'll load the pre-trained `bert-base-uncased` model for sequence classification. Since this is a binary classification task (Polarized/Not Polarized), we set `num_labels=2`.

In [ ]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now, we'll define the training arguments and the evaluation metric. We'll use macro F1 score for evaluation.

In [ ]:
# Define metrics function
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

# Define training arguments
training_args = TrainingArguments(
        output_dir=f"../models/",
        num_train_epochs=5,
        learning_rate=2e-5,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=8,
        eval_strategy="epoch",
        save_strategy="no",
        logging_steps=100,
        disable_tqdm=False
    )


Finally, we'll initialize the `Trainer` and start training.

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    data_collator=DataCollatorWithPadding(tokenizer) # Data collator for dynamic padding
)

# Train the model
trainer.train()

# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(f"Macro F1 score on validation set: {eval_results['eval_f1_macro']}")

Epoch,Training Loss,Validation Loss,F1 Macro
1,No log,0.488879,0.661950
2,No log,0.461169,0.744631
3,0.472400,0.466981,0.755739
4,0.472400,0.486929,0.747563
5,0.269600,0.491052,0.750553


Macro F1 score on validation set: 0.7505532518316396


In [ ]:
test_df = pd.read_csv(f'../data/dev_phase/subtask1/dev/{LANG}.csv')
if 'polarization' in test_df.columns:
    test_labels = test_df['polarization'].fillna(0).astype(int).tolist()
else:
    test_labels = [0] * len(test_df)

test_dataset = PolarizationDataset(
    test_df['text'].tolist(),
    test_labels,
    tokenizer,
    max_length=256
)

trainer.compute_metrics = None

prediction_output = trainer.predict(test_dataset)
logits = prediction_output.predictions
predictions_binary = np.argmax(logits, axis=1)
submission_df = pd.DataFrame()

if 'id' in test_df.columns:
    submission_df['id'] = test_df['id']
elif 'ID' in test_df.columns:
    submission_df['id'] = test_df['ID']

submission_df['polarization'] = predictions_binary

submission_df.to_csv(f"../results/predictions_1_{LANG}.csv", index=False)

# Subtask 2: Polarization Type Classification
Multi-label classification to identify the target of polarization as one of the following categories: Gender/Sexual, Political, Religious, Racial/Ethnic, or Other.
For this task we will load the data for subtask 2.

In [ ]:
df = pd.read_csv(f'../data/dev_phase/subtask2/train/{LANG}.csv')

train, val = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

train.head()

,id,text,political,racial/ethnic,religious,gender/sexual,other
3000,eng_427cea503f2e3a2b4608d26fa87a55f0,The IDF needs some B52s.,0,0,0,0,0
366,eng_98eb4278a5fb9f249149a899b8f2c4e7,Fascinating life journey leads me to ANRAurora...,0,0,0,0,0
1965,eng_0d9c500e39edc99af8c61c45db8825a2,Lazy woke excuse to justify sjw practices,1,0,0,0,0
29,eng_69ebe50510087fb8c06cd12283a88ef8,5 takeaways on Republicans first impeachment h...,0,0,0,0,0
2689,eng_787ec0b4ddda46bbb5a0bf507a31d484,And yet Dems are supposed to be the breeders o...,1,0,0,0,0


In [ ]:
# Fix the dataset class by inheriting from torch.utils.data.Dataset
class PolarizationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length # Store max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding=False, max_length=self.max_length, return_tensors='pt')

        # Ensure consistent tensor conversion for all items
        item = {key: encoding[key].squeeze() for key in encoding.keys()}
        # CHANGE THIS LINE: Use torch.float instead of torch.long for multi-label classification
        item['labels'] = torch.tensor(label, dtype=torch.float)
        return item


In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Create train and Test dataset for multilabel
train_dataset = PolarizationDataset(train['text'].tolist(), train[['gender/sexual','political','religious','racial/ethnic','other']].values.tolist(), tokenizer)
val_dataset = PolarizationDataset(val['text'].tolist(), val[['gender/sexual','political','religious','racial/ethnic','other']].values.tolist(), tokenizer)
dev_dataset = PolarizationDataset(val['text'].tolist(), val[['gender/sexual','political','religious','racial/ethnic','other']].values.tolist(), tokenizer)


In [ ]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5, problem_type="multi_label_classification") # 5 labels

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define metrics function for multi-label classification
def compute_metrics_multilabel(p):
    # Sigmoid the predictions to get probabilities
    probs = torch.sigmoid(torch.from_numpy(p.predictions))
    # Convert probabilities to predicted labels (0 or 1)
    preds = (probs > 0.5).int().numpy()
    # Compute macro F1 score
    return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

# Define training arguments
training_args = TrainingArguments(
    output_dir=f"../models/",
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=100,
    disable_tqdm=False
)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics_multilabel,  # Use the new metrics function
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train the model
trainer.train()

# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(f"Macro F1 score on validation set for Subtask 2: {eval_results['eval_f1_macro']}")

A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


Epoch,Training Loss,Validation Loss,F1 Macro
1,0.232800,0.211846,0.121212
2,0.168800,0.205062,0.162781
3,0.126700,0.237359,0.302390
4,0.106200,0.242790,0.284076
5,0.085500,0.250545,0.315667


Macro F1 score on validation set for Subtask 2: 0.31566706491437674


In [ ]:

test_df = pd.read_csv(f'../data/dev_phase/subtask2/dev/{LANG}.csv')

label_columns = ['gender/sexual', 'political', 'religious', 'racial/ethnic', 'other']
if set(label_columns).issubset(test_df.columns):
    test_df[label_columns] = test_df[label_columns].fillna(0)
    test_labels = test_df[label_columns].values.tolist()

test_dataset = PolarizationDataset(
    test_df['text'].tolist(),
    test_labels,
    tokenizer,
    max_length=256
)

trainer.compute_metrics = None


prediction_output = trainer.predict(test_dataset)

logits = torch.tensor(prediction_output.predictions)
probs = torch.sigmoid(logits)
predictions_binary = (probs > 0.5).int().numpy()

submission_df = pd.DataFrame(predictions_binary, columns=label_columns)

if 'id' in test_df.columns:
    submission_df.insert(0, 'id', test_df['id'])
elif 'ID' in test_df.columns:
    submission_df.insert(0, 'id', test_df['ID'])

submission_df.to_csv(f"../results/predictions_2_{LANG}.csv", index=False)


# Subtask 3: Manifestation Identification
Multi-label classification to classify how polarization is expressed, with multiple possible labels including Vilification, Extreme Language, Stereotype, Invalidation, Lack of Empathy, and Dehumanization.



In [ ]:
df = pd.read_csv(f'../data/dev_phase/subtask3/train/{LANG}.csv')

train, val = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

train.head()

,id,text,stereotype,vilification,dehumanization,extreme_language,lack_of_empathy,invalidation
3000,eng_427cea503f2e3a2b4608d26fa87a55f0,The IDF needs some B52s.,0,0,0,0,0,0
366,eng_98eb4278a5fb9f249149a899b8f2c4e7,Fascinating life journey leads me to ANRAurora...,0,0,0,0,0,0
1965,eng_0d9c500e39edc99af8c61c45db8825a2,Lazy woke excuse to justify sjw practices,0,1,0,0,0,0
29,eng_69ebe50510087fb8c06cd12283a88ef8,5 takeaways on Republicans first impeachment h...,0,0,0,0,0,0
2689,eng_787ec0b4ddda46bbb5a0bf507a31d484,And yet Dems are supposed to be the breeders o...,1,1,0,1,0,0


In [ ]:
# Fix the dataset class by inheriting from torch.utils.data.Dataset
class PolarizationDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length # Store max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding=False, max_length=self.max_length, return_tensors='pt')

        # Ensure consistent tensor conversion for all items
        item = {key: encoding[key].squeeze() for key in encoding.keys()}
        # CHANGE THIS LINE: Use torch.float instead of torch.long for multi-label classification
        item['labels'] = torch.tensor(label, dtype=torch.float)
        return item

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Create train and Test dataset for multilabel
train_dataset = PolarizationDataset(train['text'].tolist(), train[['vilification','extreme_language','stereotype','invalidation','lack_of_empathy','dehumanization']].values.tolist(), tokenizer)
val_dataset = PolarizationDataset(val['text'].tolist(), val[['vilification','extreme_language','stereotype','invalidation','lack_of_empathy','dehumanization']].values.tolist(), tokenizer)

In [ ]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6, problem_type="multi_label_classification") # use 6 labels

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=f"../models/",
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=100,
    disable_tqdm=False
)

# Define metrics function for multi-label classification
def compute_metrics_multilabel(p):
    # Sigmoid the predictions to get probabilities
    probs = torch.sigmoid(torch.from_numpy(p.predictions))
    # Convert probabilities to predicted labels (0 or 1)
    preds = (probs > 0.5).int().numpy()
    # Compute macro F1 score
    return {'f1_macro': f1_score(p.label_ids, preds, average='macro')}

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics_multilabel,  # Use the new metrics function
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train the model
trainer.train()

# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(f"Macro F1 score on validation set for Subtask 3: {eval_results['eval_f1_macro']}")

A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


Epoch,Training Loss,Validation Loss,F1 Macro
1,0.387500,0.368203,0.074345
2,0.330000,0.361242,0.212941
3,0.277500,0.390458,0.340134
4,0.251400,0.400231,0.346730
5,0.232000,0.412683,0.341197


Macro F1 score on validation set for Subtask 3: 0.3411965219191153


In [ ]:
test_df = pd.read_csv(f'../data/dev_phase/subtask3/dev/{LANG}.csv')

label_columns = ['stereotype', 'vilification', 'dehumanization', 'extreme_language', 'lack_of_empathy', 'invalidation']
if set(label_columns).issubset(test_df.columns):
    test_df[label_columns] = test_df[label_columns].fillna(0)
    test_labels = test_df[label_columns].values.tolist()

test_dataset = PolarizationDataset(
    test_df['text'].tolist(),
    test_labels,
    tokenizer,
    max_length=256
)

trainer.compute_metrics = None
prediction_output = trainer.predict(test_dataset)

logits = torch.tensor(prediction_output.predictions)
probs = torch.sigmoid(logits)
predictions_binary = (probs > 0.5).int().numpy()

submission_df = pd.DataFrame(predictions_binary, columns=label_columns)

if 'id' in test_df.columns:
    submission_df.insert(0, 'id', test_df['id'])
elif 'ID' in test_df.columns:
    submission_df.insert(0, 'id', test_df['ID'])

submission_df.to_csv(f"../results/predictions_3_{LANG}.csv", index=False)